In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Define model training function
def create_lstm_model(max_words, max_length):
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=128, input_length=max_length),
        SpatialDropout1D(0.2),
        LSTM(100),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Initialize an empty list to store all results across languages and categories
all_results = []

for language in ["java", "python", "pharo"]:
    df = pd.read_csv(f"/content/drive/MyDrive/Master_assignment/Seminar2/code-comment-classification/{language}/input/{language}.csv")
    lengths = [len(text.split()) for text in df['comment_sentence']]
    average_length = np.mean(lengths)
    max_length = int(average_length)

    for category in df['category'].unique():
        # Load train and test data
        data_train = df[(df['partition'] == 0) & (df['category'] == category)]
        data_test = df[(df['partition'] == 1) & (df['category'] == category)]

        X = df[df['category'] == category]['comment_sentence']
        X_train = data_train['comment_sentence']
        y_train = data_train['instance_type']
        X_test = data_test['comment_sentence']
        y_test = data_test['instance_type']

        # Tokenization
        max_words = 1000
        tokenizer = Tokenizer(num_words=max_words)
        tokenizer.fit_on_texts(X_train)

        # K-Fold Cross-Validation
        kf = KFold(n_splits=5, shuffle=True)
        accuracy_results, precision_results, recall_results, f1_results = [], [], [], []
        precision_weighted_results, recall_weighted_results, f1_weighted_results = [], [], []

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = np.array(X_train)[train_index], np.array(X_train)[val_index]
            y_train_fold, y_val_fold = np.array(y_train)[train_index], np.array(y_train)[val_index]

            # Preprocess the data
            X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train_fold), maxlen=max_length)
            X_val_pad = pad_sequences(tokenizer.texts_to_sequences(X_val_fold), maxlen=max_length)

            # Create and train model
            model = create_lstm_model(max_words, max_length)
            model.fit(X_train_pad, y_train_fold, epochs=10, batch_size=2, verbose=1)

            # Evaluate model on validation fold
            y_val_pred = (model.predict(X_val_pad) > 0.5).astype(int)
            accuracy_results.append(accuracy_score(y_val_fold, y_val_pred))
            precision_results.append(precision_score(y_val_fold, y_val_pred))
            recall_results.append(recall_score(y_val_fold, y_val_pred))
            f1_results.append(f1_score(y_val_fold, y_val_pred))
            precision_weighted_results.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
            recall_weighted_results.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
            f1_weighted_results.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

        # Train on entire training set and evaluate on test set
        X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_length)
        X_test_pad = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_length)
        y_train = np.array(y_train)
        final_model = create_lstm_model(max_words, max_length)
        final_model.fit(X_train_pad, y_train, epochs=10, batch_size=2, verbose=1)

        # Predict and calculate final test metrics
        y_test_pred = (final_model.predict(X_test_pad) > 0.5).astype(int)
        final_accuracy = accuracy_score(y_test, y_test_pred)
        final_precision = precision_score(y_test, y_test_pred)
        final_recall = recall_score(y_test, y_test_pred)
        final_f1 = f1_score(y_test, y_test_pred)
        final_precision_weighted = precision_score(y_test, y_test_pred, average='weighted')
        final_recall_weighted = recall_score(y_test, y_test_pred, average='weighted')
        final_f1_weighted = f1_score(y_test, y_test_pred, average='weighted')

        # Append k-fold and final metrics to all_results
        all_results.append({
            'Language': language,
            'Category': category,
            'Average Accuracy': np.mean(accuracy_results),
            'Average Precision': np.mean(precision_results),
            'Average Recall': np.mean(recall_results),
            'Average F1 Score': np.mean(f1_results),
            'Average Precision Weighted': np.mean(precision_weighted_results),
            'Average Recall Weighted': np.mean(recall_weighted_results),
            'Average F1 Weighted': np.mean(f1_weighted_results),
            'Final Accuracy': final_accuracy,
            'Final Precision': final_precision,
            'Final Recall': final_recall,
            'Final F1 Score': final_f1,
            'Final Precision Weighted': final_precision_weighted,
            'Final Recall Weighted': final_recall_weighted,
            'Final F1 Weighted': final_f1_weighted
        })

# Save all results to CSV
result_file_path = "/content/drive/MyDrive/Master_assignment/Seminar2/code-comment-classification/results/LSTM_RESULT_FINAL.csv"
all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv(result_file_path, index=False)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8259 - loss: 0.4572
Epoch 2/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8936 - loss: 0.2618
Epoch 3/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9322 - loss: 0.1765
Epoch 4/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9657 - loss: 0.1147
Epoch 5/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9629 - loss: 0.0846
Epoch 6/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9842 - loss: 0.0459
Epoch 7/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9839 - loss: 0.0449
Epoch 8/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9886 - loss: 0.0429
Epoch 9/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9911 - loss: 0.0281
Epoch 10/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9848 - loss: 0.0352
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


771/771 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8300 - loss: 0.4466
Epoch 2/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9046 - loss: 0.2314
Epoch 3/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9398 - loss: 0.1746
Epoch 4/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9611 - loss: 0.1103
Epoch 5/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.9696 - loss: 0.1016
Epoch 6/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9718 - loss: 0.0735
Epoch 7/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.9800 - loss: 0.0567
Epoch 8/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9883 - loss: 0.0417
Epoch 9/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9920 - loss: 0.0291
Epoch 10/10
771/771 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9897 - loss: 0.0308
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

563/563 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9130 - loss: 0.3172
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9330 - loss: 0.1979
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9646 - loss: 0.1036
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9732 - loss: 0.0650
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9914 - loss: 0.0361
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9888 - loss: 0.0370
Epoch 7/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9919 - loss: 0.0253
Epoch 8/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9928 - loss: 0.0236
Epoch 9/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9922 - loss: 0.0275
Epoch 10/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9927 - loss: 0.0268
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


563/563 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9195 - loss: 0.3069
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9291 - loss: 0.1807
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9633 - loss: 0.1054
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9856 - loss: 0.0564
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9925 - loss: 0.0262
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9913 - loss: 0.0300
Epoch 7/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9904 - loss: 0.0285
Epoch 8/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9941 - loss: 0.0206
Epoch 9/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9967 - loss: 0.0133
Epoch 10/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9949 - loss: 0.0265
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


703/703 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9130 - loss: 0.3236
Epoch 2/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9349 - loss: 0.1694
Epoch 3/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9633 - loss: 0.1009
Epoch 4/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.9818 - loss: 0.0566
Epoch 5/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9938 - loss: 0.0295
Epoch 6/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9902 - loss: 0.0308
Epoch 7/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9911 - loss: 0.0268
Epoch 8/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9909 - loss: 0.0256
Epoch 9/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.9945 - loss: 0.0190
Epoch 10/10
703/703 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9943 - loss: 0.0196
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
